# Python Wrapper for the Socrata Open Data API

## Introduction

In an effort to make the Socrata Open Data API more accessible in python, I wrote a little wrapper that helps with formatting queries and urls and app_tokens to ease the slight mess that resulted from doing it from scratch. 

**IMPORTANT: Please take some time to understand how the API works. It will really help if you know how to write SoQL queries and work with locations and datetimes correctly**

[Getting Started with the SODA Consumer API](https://dev.socrata.com/consumers/getting-started.html)

Here are some example ways of using it! 

## Imports

I am importing datetime to get data from "today" in order to speed up some of the queries.

As of this instant, the package is called soda. You will want to import the `SODA` class from the `soda` package as shown below. In addition to SODA, I am importing my application token from an external file. This is a good habit, as you can't actually see my app token this way. `settings` is just a `settings.py` file in the current direcory with the `APP_TOKEN` variable defined.

In [1]:
import datetime
import pandas as pd
# import the Socrata Open Data API
from soda import SODA

# import my application token from a file
from settings import APP_TOKEN

## Basic Setup

The only information that the SODA object really needs to be created is the base URL of the dataset you're trying to access. Let's set this up and initiate the SODA object!

In [2]:
api_url = 'https://data.seattle.gov/resource/3k2p-39jp.csv'
data = SODA(api_url)
print data.api_url

https://data.seattle.gov/resource/3k2p-39jp.csv


## Set the Token

By design, this package will not request data if you have not set your token. You can set the token one of two ways. If you would like to set your token when you instantiate your SODA object, you can pass the token in as a `token` keyword argument:
    
    data = SODA(api_url, token=APP_TOKEN)

Alternatively, we can set the token after the fact by calling `set_token(APP_TOKEN)` on the SODA object like so:

    data = SODA(api_url)
    data.set_token(APP_TOKEN)

In [3]:
data.set_token(APP_TOKEN)

## Set up the query

At this point, you could go out and get data (described later). However, you'll most likely want to do some pre-filtering and querying of the data to reduce the amount of data you get. This can be done using the `set_query` method!

For our example, we will be getting the City of Seattle 911 calls for the police department. We only want to the the most recent 10 calls from today.

At this point, you will want to familiarize yourself with [SoQL](https://dev.socrata.com/docs/queries.html). Go do that, and then come back.

Welcome back! We are going to put all of our queries into a dictionary that we can pass into the SODA object. This is structured so that the key of the dictionary is the SoQL query parameter and the value is the expression. **IMPORTANT: as of the current version, this only supports SoQL queries and NOT simple filters. Support for simple filters is on the ToDo list.** Let's dive in with an example:

In [4]:
# set the date that we'd like to look at
today = datetime.date.today()
# format the date as Floating Timestamp Datetype
# https://dev.socrata.com/docs/datatypes/floating_timestamp.html
td = today.strftime('%Y-%m-%dT%H:%M:%S')

# structure the query as a dictionary
query = dict(limit=10,
             where="event_clearance_date >= '{}'".format(td),
             order="event_clearance_date"
            )

# set the query!
data.set_query(query)

# print out the final URL of the query (using a dummy token)
data.set_token('abcd1234')
data.print_query()
print(data.url)

# reset my token back to what it should be
data.set_token(APP_TOKEN)

{'limit': 10,
 'order': 'event_clearance_date',
 'where': "event_clearance_date >= '2015-11-18T00:00:00'"}
https://data.seattle.gov/resource/3k2p-39jp.csv?$$app_token=abcd1234&$where=event_clearance_date%20>=%20%272015-11-18T00:00:00%27&$limit=10&$order=event_clearance_date


## Get some DATA!

Okay. We've created our class with the api_url and set our token. We've formatted our query so it plays nice with SoQL. Now we want to actually get some data. Up until this point, the SODA object has not made any requests to the api.

### Raw Data
We can get the data in one of several ways. When we specified the api endpoint, we specified that we would like it as a `csv` file. There are three options available: csv, xml, and json. If we use the `get_request()` method of the SODA api, you'll get the data in whatever format you requested. In this case,  we'll get a csv string.

    request = data.get_request()

In [5]:
request = data.get_request()
print request[:1000]

CAD CDW ID,CAD Event Number,General Offense Number,Event Clearance Code,Event Clearance Description,Event Clearance SubGroup,Event Clearance Group,Event Clearance Date,Hundred Block Location,District/Sector,Zone/Beat,Census Tract,Longitude,Latitude,Incident Location,Initial Type Description,Initial Type Subgroup,Initial Type Group,At Scene Time
1612516,15000402304,2015402304,280,SUSPICIOUS PERSON,SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,11/18/2015 12:07:16 AM,16XX BLOCK OF 24 AV,C,C3,7700.3004,-122.30139,47.615894,"(47.615894, -122.30139)","SUSPICIOUS PERSON, VEHICLE OR INCIDENT",SUSPICIOUS CIRCUMSTANCES,SUSPICIOUS CIRCUMSTANCES,
1612515,15000402312,2015402312,050,"BURGLARY - RESIDENTIAL, UNOCCUPIED",RESIDENTIAL BURGLARIES,BURGLARY,11/18/2015 12:09:54 AM,47XX BLOCK OF 18 AV NE,U,U1,5301.1001,-122.30834,47.663963,"(47.663963, -122.30834)",TRESPASS,TRESPASS,TRESPASS,
1612518,15000402195,2015402195,470,PARKING VIOLATION (EXCEPT ABANDONED VEHICLES),PARKING VIOLATIONS,TRAFFIC RELAT

### Pandas DataFrame

If, instead of the raw data, you'd like to see your data neatly stored as a pandas DataFrame, you can use the `get_df()` method to do just that. This will work with either the json or csv endpoints, but I tend to prefer the output that comes out from the csv formatted API endpoint. Getting the dataframe is as easy as replacing the call above with get_df() as follows:

    df = data.get_df()

In [6]:
df = data.get_df()
df.head(1).T

,0
CAD CDW ID,1612516
CAD Event Number,15000402304
General Offense Number,2015402304
Event Clearance Code,280
Event Clearance Description,SUSPICIOUS PERSON
Event Clearance SubGroup,SUSPICIOUS CIRCUMSTANCES
Event Clearance Group,SUSPICIOUS CIRCUMSTANCES
Event Clearance Date,11/18/2015 12:07:16 AM
Hundred Block Location,16XX BLOCK OF 24 AV
District/Sector,C


## Limitations

As the project stands right now, there are a handful of limitations which are really only present due to the infancy of this project. Most of these are at the top of the to-do list for implementation, and this document will be updated when these items are addressed. Limitations that I could think of are as follows, in no particular order of importance:

* Can only handle SoQL style queries, simple filtering does not work
* No support for the XML style other than get_request() (ie no pandas support)
* No support for paging through queries

## To-Do

Other than building support for the limitations listed above, I don't have much of a to-do list. Feel free to let me know what I should do!